In [1]:
from datasets import Mydataset_plip2
from torch.utils.data import DataLoader
from load_model import model_fusion
import torch
import pandas as pd
import torch.nn.functional as F
from transformers import CLIPProcessor, CLIPModel
import numpy as np

In [2]:
data = pd.read_csv("internal_corhort.csv")

In [3]:
data.head(1) # the protoype  embedding get from step1

,name,case,kind,description,disease,prototype,pro_num
0,a21-1884_brain0.csv,a21-1884,brain,Scattered punctate hemorrhage is observed unde...,The hemorrhage under the scalp,"{830: 1259, 555: 567, 786: 504, 535: 497, 857:...",80


In [4]:
dd = Mydataset_plip2("internal_corhort.csv", randomm=False)
trainloader = DataLoader(dd, batch_size=1, shuffle=False,drop_last=False)

In [5]:
model = model_fusion(depth=2,noise_ratio=0.0, gate=True,num_em=True)
model.load_state_dict(torch.load("fusion_checkpoint.pth",map_location="cpu"))

<All keys matched successfully>

In [6]:
model.eval()
model.cuda()

fusionblock2(
  (text_model): CLIPModel(
    (shared_parameters): ModuleDict()
    (text_model): CLIPTextTransformer(
      (invertible_adapters): ModuleDict()
      (embeddings): CLIPTextEmbeddings(
        (token_embedding): Embedding(49408, 512)
        (position_embedding): Embedding(77, 512)
      )
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0): CLIPEncoderLayer(
            (self_attn): CLIPAttention(
              (k_proj): Linear(
                in_features=512, out_features=512, bias=True
                (loras): ModuleDict()
              )
              (v_proj): Linear(
                in_features=512, out_features=512, bias=True
                (loras): ModuleDict()
              )
              (q_proj): Linear(
                in_features=512, out_features=512, bias=True
                (loras): ModuleDict()
              )
              (out_proj): Linear(in_features=512, out_features=512, bias=True)
              (prefix_tuning): PrefixTuni

In [7]:
# If you have a set of candidate diagnoses, we can use the diagnosis of data as an example of a candidate in this context.
for it, (tokens_disease,tokens_description,img,img_num) in enumerate(trainloader):
    tokens_description["input_ids"] = tokens_description["input_ids"].squeeze(1).cuda(non_blocking=True)
    tokens_description["attention_mask"] = tokens_description["attention_mask"].squeeze(1).cuda(non_blocking=True)
    tokens_disease["input_ids"] = tokens_disease["input_ids"].squeeze(1).cuda(non_blocking=True)
    tokens_disease["attention_mask"] = tokens_disease["attention_mask"].squeeze(1).cuda(non_blocking=True)
    _, _, _, img_text_out, disease, _ = model(img_p=img.cuda(non_blocking=True),text=tokens_description,img_num=img_num.cuda(non_blocking=True),disease=tokens_disease)
    score = img_text_out.cpu().squeeze() @ disease.cpu().squeeze().t()

    break

In [8]:
print(score) # Return the similarity between the fused embedding of the multimodal input and the target diagnosis!

tensor(0.9173, grad_fn=<DotBackward0>)


In [9]:
# If you do not have any candidate diagnoses, as an exsample, we provide commonly used diagnoses in forensic pathology here.

In [10]:
with open("disease.txt", "r") as file:
    my_list = [line.strip() for line in file]
tokenizer = CLIPProcessor.from_pretrained("vinid/plip").tokenizer
token_disease = tokenizer(my_list, padding="max_length", max_length=77, return_tensors="pt")
token_disease["input_ids"] = token_disease["input_ids"].cuda(non_blocking=True)
token_disease["attention_mask"] = token_disease["attention_mask"].cuda(non_blocking=True)
disease_model = CLIPModel.from_pretrained("vinid/plip")
disease_model.eval()
disease_model.cuda()
disease_rep = disease_model.get_text_features(**token_disease)
disease_rep = F.normalize(disease_rep, dim=-1).cpu()
for it, (tokens_disease,tokens_description,img,img_num) in enumerate(trainloader):
    score_list=[]
    model.eval()
    with torch.no_grad():
        tokens_description["input_ids"] = tokens_description["input_ids"].squeeze(1).cuda(non_blocking=True)
        tokens_description["attention_mask"] = tokens_description["attention_mask"].squeeze(1).cuda(non_blocking=True)
        for ind, (t_disease,t_attention) in enumerate(zip(token_disease["input_ids"],token_disease["attention_mask"])):
            _, _, _, img_text_out, disease, attention_score = model(img_p=img.cuda(non_blocking=True),
                                                                           text=tokens_description,
                                                                           img_num=img_num.cuda(non_blocking=True),
                                                                           disease={"input_ids":t_disease.unsqueeze(0),"attention_mask":t_attention.unsqueeze(0)})
            score = img_text_out.cpu().squeeze() @ disease_rep[ind].t()
            score_list.append(score)

        score_array = np.array(score_list)


        disease_name_max = [my_list[i] for i in score_array.argsort()[-1:][::-1].tolist()]  # top1
        disease_name_top2 = [my_list[i] for i in score_array.argsort()[-2:][::-1].tolist()] # top2 
        disease_name_top3 = [my_list[i] for i in score_array.argsort()[-3:][::-1].tolist()]  # top3 
    break


In [11]:
disease_name_max # best answer!

['The hemorrhage under the scalp']

In [12]:
disease_name_top2

['The hemorrhage under the scalp', 'Cerebral hemorrhage']

In [13]:
disease_name_top3

['The hemorrhage under the scalp',
 'Cerebral hemorrhage',
 'Cerebral congestion']

In [14]:
data.loc[0,"disease"] # the groudtruth 

'The hemorrhage under the scalp'